# Lectura de los archivos

In [40]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
from collections import defaultdict

In [41]:
bac_files = 'data/BAC/'

Primero debemos limpiar los datos que tienen entidades HTML esto lo vamos a manejar con bs4 que nos permite trabajar con este tipo de entidades sin eliminarlo manualmente

In [38]:
def clean_and_overwrite_xml(filepath):
    # Intentar abrir el archivo con una codificación más flexible como 'latin-1'
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            content = file.read()
    except UnicodeDecodeError:
        with open(filepath, 'r', encoding='latin-1') as file:
            content = file.read()

    # Usar BeautifulSoup para manejar entidades HTML dentro del XML
    soup = BeautifulSoup(content, 'xml')

    # Convertir el contenido limpio a una cadena de texto
    cleaned_content = str(soup)

    # Sobrescribir el archivo original con el contenido limpio
    with open(filepath, 'w', encoding='utf-8') as file:
        file.write(cleaned_content)

In [ ]:
# Ruta a la carpeta donde están almacenados los archivos XML
ruta_archivos = 'data/BAC/'

# Procesar todos los archivos XML en la carpeta
for archivo in os.listdir(ruta_archivos):
    if archivo.endswith('.xml'):
        archivo_completo = os.path.join(ruta_archivos, archivo)
        clean_and_overwrite_xml(archivo_completo)
        print(f"Archivo procesado: {archivo}")

Realizamos el procesamiento de los datos de BAC, donde tomamos las columnas como el nombre del archivo es decir, si el nombre es 4334776.male.24.Engineering.Aquarius, entonces las columnas son

* id: 4334776
* gender: male
* age: 24
* profession: Engineering
* zodiac_sign: Aguarius

Para manejar las entradas de los blogs decidimos agregar date, blog_content y number_of_posts. Esto se hizo para no perder informacion de las entradas realizadas por el usuario, de tal forma que, seria una fila en el dataframe por cada fecha que realizo una entrada el usuario. Si hay varias entradas por una fecha, se concatenan con un caracter definido por nosotros como ***POST** *  para identificar en donde empieza la siguiente entrada. Ademas, decidimos crear una funcion para convertir la fecha a formato que python identifique como YYYY-MM-DD

In [ ]:
# Dictionary to convert month names to numbers
months = {
    "January": "01", "February": "02", "March": "03", "April": "04", 
    "May": "05", "June": "06", "July": "07", "August": "08", 
    "September": "09", "October": "10", "November": "11", "December": "12"
}

# Function to convert date from format DD,Month,YYYY to YYYY-MM-DD
def convert_date_to_yyyymmdd(date_str):
    try:
        # Separate the date into day, month, and year
        day, month, year = date_str.split(',')

        # Remove extra spaces and convert the month name to a number
        day = day.strip().zfill(2)  # Ensure the day has two digits
        month = months[month.strip()]
        year = year.strip()

        # Format the date as YYYY-MM-DD
        formatted_date = f"{year}-{month}-{day}"
        
        return formatted_date
    except Exception as e:
        print(f"Error converting date {date_str}: {e}")
        return None



In [ ]:
# Función para leer el archivo XML, extraer las publicaciones agrupadas por fecha y devolver un DataFrame
def bac_process(filepath):
    try:
        # Extraer atributos del nombre del archivo: id, gender, age, profession, zodiac_sign
        filename = os.path.basename(filepath).replace('.xml', '')
        columns = filename.split('.')

        if len(columns) != 5:
            raise ValueError(f"El archivo {filename} no tiene el formato adecuado.")

        id_, gender, age, profession, zodiac_sign = columns

        # Intentar abrir el archivo con una codificación más flexible como 'latin-1'
        try:
            with open(filepath, 'r', encoding='utf-8') as file:
                content = file.read()
        except UnicodeDecodeError:
            with open(filepath, 'r', encoding='latin-1') as file:
                content = file.read()

        # Usar BeautifulSoup para manejar el XML
        soup = BeautifulSoup(content, 'xml')

        # Extraer todas las dates y las publicaciones
        dates = [date.text for date in soup.find_all('date')]
        publicaciones = [post.text.strip() for post in soup.find_all('post')]

        # Agrupar las publicaciones por su fecha
        post_by_date = defaultdict(list)
        for fecha, publicacion in zip(dates, publicaciones):
            post_by_date[fecha].append(publicacion)

        # Preparar los datos para el DataFrame
        data = []
        for fecha, posts in post_by_date.items():
            # Concatenar las publicaciones con 'POST' y contar el número de publicaciones
            blog_content = 'POST'.join(posts)
            number_of_posts = len(posts)
            
            formatted_date = convert_date_to_yyyymmdd(fecha)

            # Agregar una fila al conjunto de datos
            data.append([id_, gender, age, profession, zodiac_sign, formatted_date, blog_content, number_of_posts])

        # Crear un DataFrame para este archivo
        df = pd.DataFrame(data, columns=['id', 'gender', 'age', 'profession', 'zodiac_sign', 'date', 'blog_content', 'number_of_posts'])

        return df

    except Exception as e:
        print(f"Error al procesar {filepath}: {e}")
        return pd.DataFrame()

# Ruta a la carpeta donde están almacenados los archivos XML
ruta_archivos = 'data/BAC/'

# Lista para almacenar los DataFrames de todos los archivos
df_list = []

# Procesar todos los archivos XML en la carpeta
for archivo in os.listdir(ruta_archivos):
    if archivo.endswith('.xml'):
        archivo_completo = os.path.join(ruta_archivos, archivo)
        df = bac_process(archivo_completo)
        if not df.empty:
            df_list.append(df)

# Concatenar todos los DataFrames en uno solo
df_final = pd.concat(df_list, ignore_index=True)

# Mostrar el DataFrame final
df_final

In [45]:
df_final.to_csv('bac_unified.csv', index=False)


In [46]:
df = pd.read_csv("bac_unified.csv")

In [ ]:
df